In [1]:
# Extract SOC information from MOLPRO outputs for atoms
# This version to handle multiple terms with the same label
# KKI 6/22/23
import re, sys, glob, subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.cluster import KMeans

#sys.path.insert(0, '../karlib')
import chem_subs as chem
import molpro_subs as mpr

pd.set_option('display.max_rows', None)
np.set_printoptions(suppress=True)

In [2]:
# Excel spreadsheet of experimental levels from https://physics.nist.gov/PhysRefData/ASD/levels_form.html
#   Download as CSV; paste into a column in Excel; use Data -> Text to Columns -> Delimited -> Comma
#   Rename that worksheet with a name like "Fe" or "Fe+"
# Note that experimental levels might not be listed by increasing energy
xl_expt = 'exptl_levels.xlsx'
xl = pd.ExcelFile(xl_expt, engine='openpyxl')

### Select atom and parity of interest

In [3]:
atom = 'Ni'  # a name like "Fe" or "Fe+"
parity = 'even'  #  choose 'even' or 'odd' or 'both'
keep_metastable = False  # whether to discard levels above the ionization limit

### Select energy maximum for experimental terms

In [4]:
# In case of errors, try making this larger or smaller to match the theoretical calculation
termcut = 25000  # discard terms that lack levels below this energy (cm-1)

In [5]:
Ecol = 'Level (cm-1)'  # the exptl energy column
# display formatting
fmt = {'Eshift': '{:.1f}', Ecol: '{:.3f}', 'Pct': '{:.3f}', 'degen': '{:.0f}'}
for col in ['J', 'Ecalc', 'E_dif', 'Erel', 'Eshift', 'err', 'Eterm', 'Elev', 'wmean', 'wstds', 
            'uwmean', 'uwstds']:
    fmt[col] =  fmt['Eshift']

In [6]:
if atom not in xl.sheet_names:
    print(f'No experimental data sheet for {atom}!')
else:
    dfexpt = pd.read_excel(xl, atom, engine='openpyxl')
    # Delete any ionization limit
    ilim = dfexpt[dfexpt.Term == 'Limit'].index.min()
    # delete the "Limit" row
    dfexpt = dfexpt[dfexpt.Term != 'Limit']
    if not keep_metastable:
        # delete everything past the Limit
        n1 = len(dfexpt)
        dfexpt = dfexpt.truncate(after=ilim)
        n2 = len(dfexpt)
        if n2 < n1:
            print(f'Discarded {n1-n2} metastable states')
    dfeven = dfexpt[~dfexpt.Term.str.contains('\*$')].copy()
    dfodd = dfexpt[dfexpt.Term.str.contains('\*$')].copy()
    print(f'{len(dfexpt)} experimental levels ({len(dfeven)} even and {len(dfodd)} odd) for {atom} read from "{xl_expt}"')
    # Select by parity
    if parity == 'even':
        # discard odd levels ('Term' field ends with '*')
        dfexpt = dfeven.copy()
    elif parity == 'odd':
        dfexpt = dfodd.copy()
    print(f'{len(dfexpt)} levels are of parity "{parity}"')
# sort by term label and then by energy
dfexpt = dfexpt.sort_values(['Term', Ecol])
display(dfexpt)

Discarded 2 metastable states
285 experimental levels (128 even and 157 odd) for Ni read from "exptl_levels.xlsx"
128 levels are of parity "even"


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference
6,3d9.(2D).4s,1D,2,NaN,3409.937,NaN,NaN,1.01297,89 : 9 3d9.(2D).4s ...,NaN
7,3d8.(1D).4s2,1D,2,NaN,13521.347,NaN,NaN,1.14300,77 : 19 3d8.(3P).4s2 ...,NaN
144,3d8.4s.(2F).5s,1F,3,NaN,54251.308,NaN,NaN,NaN,64 : 20 3d8.4s.(2F).5s ...,NaN
12,3d8.(1G).4s2,1G,4,NaN,22102.325,NaN,NaN,0.99000,97,NaN
8,3d10,1S,0,NaN,14728.840,NaN,NaN,NaN,94,NaN
121,3d8.(1S).4s2,1S,0,NaN,50276.321,NaN,NaN,NaN,90,NaN
104,3d9.(2D<5/2>).4d,2[1/2],1,NaN,48953.316,NaN,NaN,1.92000,99,NaN
105,3d9.(2D<5/2>).4d,2[1/2],0,NaN,49610.345,NaN,NaN,NaN,74 : 22 3d9.(2D<5/2>).4d ...,NaN
126,3d9.(2D<3/2>).4d,2[1/2],1,NaN,50536.703,NaN,NaN,1.54000,99,NaN
127,3d9.(2D<3/2>).4d,2[1/2],0,NaN,51457.250,NaN,NaN,NaN,67 : 19 3d9.(2D<5/2>).4d ...,NaN


In [7]:
# Print any unrecognized term labels
for trm in dfexpt.Term:
    try:
        chem.possible_J_from_ASD_label(trm)
    except:
        print(trm)

In [8]:
# Assign unique labels to experimental terms
#   levels from ASD are usually grouped by term, but not always
uterm = []  # unique terms
uidx = []   # list of list of rows that belong to each unique term
jvals = []  # list of J values for current term
idx = []    # list of rows for current term
curterm = None
problems = 0

for irow, row in dfexpt.iterrows():
    trm = row.Term
    J = chem.halves_to_float(row.J)
    if (trm != curterm) or (len(jvals) == 0):
        if len(jvals) > 0:
            print(f'Unused J values for term {curterm} before irow = {irow}: {jvals}')
            problems += 1
        # close out previous term
        if idx:
            uidx.append(idx)
        # start a new term
        curterm = trm
        uterm.append(trm)
        idx = [irow]   # row numbers for current term
        jvals = list(chem.possible_J_from_ASD_label(trm))
        if J in jvals:
            jvals.remove(J)
        else:
            print(f'Unexpected J = {J} not in {jvals} for term {curterm} near E ~ {row[Ecol]}')
            problems += 1
    else:
        # continuation of term
        idx.append(irow)
        if J in jvals:
            jvals.remove(J)
        else:
            print(f'Unexpected J = {J} for term {curterm} near E ~ {row[Ecol]}')
            problems += 1
if len(jvals):
    # leftover rows from last term
    uidx.append(idx)
dfexpt['uTerm'] = None
eterm = chem.enumerative_prefix(uterm)
for iterm, idx in enumerate(uidx):
    for i in idx:
        dfexpt.at[i, 'uTerm'] = eterm[iterm]
if problems:
    print('*** uTerm groupings may be wrong!  ***')
    display(dfexpt)

Unused J values for term 2[5/2] before irow = 114: [3.0]
*** uTerm groupings may be wrong!  ***


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,uTerm
6,3d9.(2D).4s,1D,2,NaN,3409.937,NaN,NaN,1.01297,89 : 9 3d9.(2D).4s ...,NaN,(1)1D
7,3d8.(1D).4s2,1D,2,NaN,13521.347,NaN,NaN,1.14300,77 : 19 3d8.(3P).4s2 ...,NaN,(2)1D
144,3d8.4s.(2F).5s,1F,3,NaN,54251.308,NaN,NaN,NaN,64 : 20 3d8.4s.(2F).5s ...,NaN,1F
12,3d8.(1G).4s2,1G,4,NaN,22102.325,NaN,NaN,0.99000,97,NaN,1G
8,3d10,1S,0,NaN,14728.840,NaN,NaN,NaN,94,NaN,(1)1S
121,3d8.(1S).4s2,1S,0,NaN,50276.321,NaN,NaN,NaN,90,NaN,(2)1S
104,3d9.(2D<5/2>).4d,2[1/2],1,NaN,48953.316,NaN,NaN,1.92000,99,NaN,(1)2[1/2]
105,3d9.(2D<5/2>).4d,2[1/2],0,NaN,49610.345,NaN,NaN,NaN,74 : 22 3d9.(2D<5/2>).4d ...,NaN,(1)2[1/2]
126,3d9.(2D<3/2>).4d,2[1/2],1,NaN,50536.703,NaN,NaN,1.54000,99,NaN,(2)2[1/2]
127,3d9.(2D<3/2>).4d,2[1/2],0,NaN,51457.250,NaN,NaN,NaN,67 : 19 3d9.(2D<5/2>).4d ...,NaN,(2)2[1/2]


In [9]:
# Select terms by energy
lowTerms = []
for term, grp in dfexpt.groupby('uTerm'):
    if (grp[Ecol] < termcut).any():
        lowTerms.append(term)
print(f'There are {len(lowTerms)} assigned terms with levels below {termcut} cm-1')
# discard high terms
dfexpt = dfexpt[dfexpt.uTerm.isin(lowTerms)].copy()
print('Re-labeling terms (uTerm) for this limited set of terms')
uterms = chem.update_enumerative_prefix(dfexpt.uTerm)
dfexpt['uTerm'] = uterms
nlevx = len(dfexpt)
print(f'There are {nlevx} levels associated with these terms of interest')
# parse 'Term' column to get simplified term labels
dfexpt['Tlbl'] = dfexpt.Term.apply(chem.strip_enumerative_prefix)
# Convert experimental 'J' and 'Level' to floats
dfexpt[Ecol] = dfexpt[Ecol].astype(float)
dfexpt['J'] = dfexpt['J'].apply(chem.halves_to_float)
# add degeneracy = 2J+1
dfexpt['degen'] = 2 * dfexpt.J + 1
# sort by energy
dfexpt = dfexpt.sort_values(Ecol)
display(dfexpt.style.format(fmt))  

There are 7 assigned terms with levels below 25000 cm-1
Re-labeling terms (uTerm) for this limited set of terms
There are 13 levels associated with these terms of interest


,Configuration,Term,J,Prefix,Level (cm-1),Suffix,Uncertainty (cm-1),Lande,Leading percentages,Reference,uTerm,Tlbl,degen
0,3d8.(3F).4s2,3F,4.0,nan,0.000,nan,nan,1.249650,97,L11151,3F,3F,9
3,3d9.(2D).4s,3D,3.0,nan,204.787,nan,nan,1.333540,99,nan,3D,3D,7
4,3d9.(2D).4s,3D,2.0,nan,879.816,nan,nan,1.151050,90 : 9 3d9.(2D).4s 1D,nan,3D,3D,5
1,3d8.(3F).4s2,3F,3.0,nan,1332.164,nan,nan,1.082800,97,nan,3F,3F,7
5,3d9.(2D).4s,3D,1.0,nan,1713.087,nan,nan,0.498040,99,nan,3D,3D,3
2,3d8.(3F).4s2,3F,2.0,nan,2216.550,nan,nan,0.669560,96,nan,3F,3F,5
6,3d9.(2D).4s,1D,2.0,nan,3409.937,nan,nan,1.012970,89 : 9 3d9.(2D).4s 3D,nan,(1)1D,1D,5
7,3d8.(1D).4s2,1D,2.0,nan,13521.347,nan,nan,1.143000,77 : 19 3d8.(3P).4s2 3P,nan,(2)1D,1D,5
8,3d10,1S,0.0,nan,14728.840,nan,nan,nan,94,nan,1S,1S,1
9,3d8.(3P).4s2,3P,2.0,nan,15609.844,nan,nan,1.356000,78 : 18 3d8.(1D).4s2 1D,nan,3P,3P,5


### Take assignments at face value, i.e., apply eq. (1)

In [10]:
# No theoretical calculations are needed to use eq. (1)
xterms = []  # list of term labels
eterms = []  # list of term energies
for term in dfexpt.uTerm:
    if term not in xterms:
        xterms.append(term)
for Term in xterms:
    subdf = dfexpt[dfexpt.uTerm == Term]
    emean = np.dot(subdf.degen, subdf[Ecol]) / subdf.degen.sum()
    eterms.append(emean)
dfeq1 = pd.DataFrame({'Term': xterms, 'Eterm': eterms}).sort_values('Eterm').reset_index(drop=True)
print('Term energies (cm-1) using eq. (1)')
display(dfeq1.style.format(fmt))
SOC1 = -1 * np.round(dfeq1.at[0, 'Eterm'], 3)
lowterm = dfeq1.at[0, 'Term']
print(f'The term of lowest energy is \t{lowterm} \twith SOC1 = {SOC1} cm-1')
levterm = dfexpt.uTerm.values[0]
if levterm != lowterm:
    # The lowest term is not the leading term in the lowest level
    SOC1 = -1 * np.round(dfeq1[dfeq1.Term == levterm]['Eterm'].values[0], 3)
    print(f'The lowest level belongs to \t{levterm} \twith SOC1 = {SOC1} cm-1')

Term energies (cm-1) using eq. (1)


,Term,Eterm
0,3D,731.5
1,3F,971.8
2,(1)1D,3409.9
3,(2)1D,13521.3
4,1S,14728.8
5,3P,15696.5
6,1G,22102.3


The term of lowest energy is 	3D 	with SOC1 = -731.457 cm-1
The lowest level belongs to 	3F 	with SOC1 = -971.805 cm-1


### Specify Molpro SO-CI output file

In [11]:
#fsoc = 'fe_15Q21T_ctzdk_x2c.pro'
#fsoc = 'fe_ci_15Q7T_c5zdk_x2c.pro'
#fsoc = 'fe_15Q7T_ctzdk_x2c.pro'
fsoc = '../UMemphis/Ni15T20S-cc-pVTZ-DK.out'
#fsoc = '../Umemphis/Au-cc-tz-pp.out'

print(f'Reading MOLPRO file "{fsoc}"')
compAtom = mpr.stoichiometry(fsoc)
charge = mpr.total_charge(fsoc, verbose=True)
print(f'The atom is {compAtom} with charge {charge}')
# check for consistency with the experimental data that were read
if charge > 0: 
    compAtom += '+'
elif charge < 0:
    compAtom += '-'
if abs(charge) > 1:
    compAtom += f'{abs(charge)}'
        
if compAtom != atom:
    print(f'*** exptl atom = {atom} is different ***')
PG = mpr.read_compgroup(fsoc)
print(f'The computational point group is {PG}')

Reading MOLPRO file "../UMemphis/Ni15T20S-cc-pVTZ-DK.out"
The atom is Ni with charge 0.0
The computational point group is Ci


In [12]:
SOCI = mpr.fullmatSOCI(fsoc, atom=True)

Computational group = Ci
CASSCF states:
    20 Singlet
    15 Triplet


In [13]:
SOCraw = SOCI.vals.min()
print(f'From lowest level and lowest uncoupled energy, raw theoretical SOCraw = {SOCraw:.3f} cm-1')

From lowest level and lowest uncoupled energy, raw theoretical SOCraw = -1004.948 cm-1


In [14]:
def term_energy_from_levels(df, term, Ecol):
    # Given a DataFrame with the right columns ['J', 'termwt', Ecol],
    #   where 'Ecol' is the header for the column of level energies,
    # Return the term's average energy as derived from the levels
    global SOCI
    # find index for term 'term'
    iterm = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    termwt = np.array([twt[iterm] for twt in df.termwt])
    degen = 2 * df.J.values + 1
    dweight = degen * termwt  # total weight, including degeneracies
    Eterm = np.dot(df[Ecol], dweight) / dweight.sum()
    return Eterm

## Select the term of interest

In [15]:
target = '3F'  # the term of interest
dflevel = SOCI.assign_atomic_J()
if target not in SOCI.dfterm.Term.values:
    print(f'*** Term {target} is not present.  Choose another! ***')
    1/0
else:
    Eterm = term_energy_from_levels(dflevel, target, 'Erel')
    print(f'Using degenerated averages, energy of {target} term = {Eterm:.1f} cm-1')
    SOCth = -Eterm
    print(f'SOCth = {SOCth:.3f} cm-1')

Assigning J using 13 clusters/levels
Using degenerated averages, energy of 3F term = 1004.9 cm-1
SOCth = -1004.946 cm-1


In [16]:
# Match experimental levels with corresponding theoretical
# First pass:  sort by energy, match by J

warnThresh = 1000  # highlight errors larger than this (cm-1)
dfdiff = dfexpt.copy().sort_values(Ecol)
dfdiff['calcTerm'] = ''
dfdiff['Ecalc'] = np.nan
print(f'Matching theoretical levels from {fsoc} with experimental')
print('   Matching levels by J and energy')
idx = list(dflevel.index)  # list of computed levels
termwt = []  # for arrays of term weights (from theory)
for i, row in dfexpt.iterrows():
    while idx:
        # there are theoretical levels that have not been matched to exptl
        for j in idx.copy():
            if float(row.J) != float(dflevel.at[j, 'J']):
                # values of J must be equal
                continue
            # J matches
            dfdiff.at[i, 'calcTerm'] = dflevel.at[j, 'Lead']
            dfdiff.at[i, 'Ecalc'] = dflevel.at[j, 'Erel']
            termwt.append(dflevel.at[j, 'termwt'])
            idx.remove(j)
            break
        else:
            print('Failed to assign any theoretical level to this exptl!')
            display(row.to_frame().T)
        break
dfdiff['err'] = np.round(dfdiff.Ecalc - dfdiff[Ecol], 2)
dfdiff['termwt'] = termwt
dfdiff = dfdiff[['Configuration', 'uTerm', 'J', 'Leading percentages', Ecol, 'calcTerm', 'Ecalc', 'err', 'termwt']]
display(dfdiff.drop('termwt', axis=1).style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']]).apply(lambda x: (x != dfdiff['uTerm']).map({True: "background-color: red; color: white", False: ""}),
                                                   subset=['calcTerm']).format(fmt))
print('Disagreements in term assignments are highlighted')
print(f'Errors > {warnThresh} cm-1 are highlighted')

Matching theoretical levels from ../UMemphis/Ni15T20S-cc-pVTZ-DK.out with experimental
   Matching levels by J and energy


,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err
0,3d8.(3F).4s2,3F,4.0,97,0.000,3F,0.0,0.0
3,3d9.(2D).4s,3D,3.0,99,204.787,3D,1057.4,852.6
4,3d9.(2D).4s,3D,2.0,90 : 9 3d9.(2D).4s 1D,879.816,3D,1727.6,847.8
1,3d8.(3F).4s2,3F,3.0,97,1332.164,3F,1334.1,2.0
5,3d9.(2D).4s,3D,1.0,99,1713.087,3D,2583.7,870.6
2,3d8.(3F).4s2,3F,2.0,96,2216.550,3F,2240.3,23.7
6,3d9.(2D).4s,(1)1D,2.0,89 : 9 3d9.(2D).4s 3D,3409.937,(1)1D,4199.1,789.1
7,3d8.(1D).4s2,(2)1D,2.0,77 : 19 3d8.(3P).4s2 3P,13521.347,3P,14997.0,1475.7
8,3d10,1S,0.0,94,14728.840,3P,16904.3,2175.5
9,3d8.(3P).4s2,3P,2.0,78 : 18 3d8.(1D).4s2 1D,15609.844,(2)1D,16885.9,1276.1


Disagreements in term assignments are highlighted
Errors > 1000 cm-1 are highlighted


In [17]:
# Second pass

# Where the term assignments disagree, is there substantial weight in the exptl term?
min_weight = 0.3  # don't allow any assignment with a smaller weight
dfdiff['OK'] = (dfdiff.calcTerm == dfdiff.uTerm)
#display(dfdiff[dfdiff.OK == False].style.format(fmt))
for irow, row in dfdiff.iterrows():
    if row.uTerm != row.calcTerm:
        display(row.to_frame().T.style.format(fmt))
        weights = {t: np.round(w, 3) for t, w in zip(SOCI.dfterm.Term.values, row.termwt)}
        wx = weights[row.uTerm]
        wth = weights[row.calcTerm]
        print(f'Conflict in leading term: theor = {row.calcTerm} with weight {wth}, exptl = {row.uTerm} with weight {wx}')
        if wx >= min_weight:
            print('\tconflict is nominal')
            dfdiff.at[irow, 'OK'] = True
        else:
            print('\tthis is not a match')

,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,termwt,OK
7,3d8.(1D).4s2,(2)1D,2.0,77 : 19 3d8.(3P).4s2 3P,13521.347,3P,14997.0,1475.7,[0.0030273 0.00002122 0.00006736 0.4884118 0.50847233 0. 0. ],False


Conflict in leading term: theor = 3P with weight 0.508, exptl = (2)1D with weight 0.488
	conflict is nominal


,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,termwt,OK
8,3d10,1S,0.0,94,14728.840,3P,16904.3,2175.5,[0. 0. 0. 0.00000043 0.99683377 0. 0.00316579],False


Conflict in leading term: theor = 3P with weight 0.997, exptl = 1S with weight 0.003
	this is not a match


,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,termwt,OK
9,3d8.(3P).4s2,3P,2.0,78 : 18 3d8.(1D).4s2 1D,15609.844,(2)1D,16885.9,1276.1,[0.00239119 0.00001487 0.00005662 0.50615843 0.49137889 0. 0. ],False


Conflict in leading term: theor = (2)1D with weight 0.506, exptl = 3P with weight 0.491
	conflict is nominal


,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,termwt,OK
11,3d8.(3P).4s2,3P,0.0,96,16017.306,1S,45445.0,29427.7,[0. 0. 0. 0. 0.00316579 0. 0.99683421],False


Conflict in leading term: theor = 1S with weight 0.997, exptl = 3P with weight 0.003
	this is not a match


In [18]:
# Third pass

# Try to resolve term-label conflicts by swapping assignments
#    this relies on the exptl term actually leading some other term
for lbl, grp in dfdiff.groupby(['OK', 'J']):
    if lbl[0]:
        continue
    if len(grp) > 1:
        # a swap is possible
        subdf = grp.copy()  # contains the rows with conflicting term labels and same "J"
        display(subdf.style.format(fmt))
        for i, row in subdf.iterrows():
            if dfdiff.at[i, 'OK']:
                # this row has already been fixed
                continue
            #display(row.to_frame().T.style.format(fmt))
            if (row.calcTerm in subdf.uTerm.values) and (row.uTerm in subdf.calcTerm.values):
                # swap is possible
                targTerm = row.calcTerm
                j = subdf[subdf.uTerm == targTerm].index.values[0]  # there should be only one
                print(f'Swapping assignments for rows {i} and {j}')
                if dfdiff.at[j, 'OK']:
                    print(f'\t*** row {j} already assigned ***')
                # make the swap
                for col in ['calcTerm', 'Ecalc', 'termwt']:
                    dfdiff.at[i, col] = subdf.at[j, col]
                    dfdiff.at[j, col] = subdf.at[i, col]
                    dfdiff.at[i, 'OK'] = True
                    dfdiff.at[j, 'OK'] = True
# recalculate the "err" column
dfdiff['err'] = dfdiff.Ecalc - dfdiff[Ecol]

,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,termwt,OK
8,3d10,1S,0.0,94,14728.840,3P,16904.3,2175.5,[0. 0. 0. 0.00000043 0.99683377 0. 0.00316579],False
11,3d8.(3P).4s2,3P,0.0,96,16017.306,1S,45445.0,29427.7,[0. 0. 0. 0. 0.00316579 0. 0.99683421],False


Swapping assignments for rows 8 and 11


In [19]:
display(dfdiff.drop('termwt', axis=1).style.apply(lambda x: ["background: yellow" if abs(v) > warnThresh else "" for v in x], 
              subset=pd.IndexSlice[['err']]).apply(lambda x: (x != dfdiff['uTerm']).map({True: "background-color: red; color: white", False: ""}),
                                                   subset=['calcTerm']).format(fmt))
print('Disagreements in term assignments are highlighted')
print(f'Errors > {warnThresh} cm-1 are highlighted')

,Configuration,uTerm,J,Leading percentages,Level (cm-1),calcTerm,Ecalc,err,OK
0,3d8.(3F).4s2,3F,4.0,97,0.000,3F,0.0,0.0,True
3,3d9.(2D).4s,3D,3.0,99,204.787,3D,1057.4,852.6,True
4,3d9.(2D).4s,3D,2.0,90 : 9 3d9.(2D).4s 1D,879.816,3D,1727.6,847.8,True
1,3d8.(3F).4s2,3F,3.0,97,1332.164,3F,1334.1,2.0,True
5,3d9.(2D).4s,3D,1.0,99,1713.087,3D,2583.7,870.6,True
2,3d8.(3F).4s2,3F,2.0,96,2216.550,3F,2240.3,23.7,True
6,3d9.(2D).4s,(1)1D,2.0,89 : 9 3d9.(2D).4s 3D,3409.937,(1)1D,4199.1,789.1,True
7,3d8.(1D).4s2,(2)1D,2.0,77 : 19 3d8.(3P).4s2 3P,13521.347,3P,14997.0,1475.7,True
8,3d10,1S,0.0,94,14728.840,1S,45445.0,30716.2,True
9,3d8.(3P).4s2,3P,2.0,78 : 18 3d8.(1D).4s2 1D,15609.844,(2)1D,16885.9,1276.1,True


Disagreements in term assignments are highlighted
Errors > 1000 cm-1 are highlighted


In [20]:
# Use experimental level energies via eq. (2)
Eterm = term_energy_from_levels(dfdiff, target, 'Level (cm-1)')
SOC2 = -Eterm
print(f'Using experimental levels and eq. (2) for term {target}, SOC2 = {SOC2:.1f} cm-1')

Using experimental levels and eq. (2) for term 3F, SOC2 = -994.4 cm-1


In [21]:
def term_distrib(term, df):
    # return the weights (including 2J+1) of term in levels
    global SOCI
    itarget = SOCI.dfterm[SOCI.dfterm.Term == term].index[0]
    wt = [twt[itarget] for twt in df.termwt]  # without 2J+1 weighting
    wt = wt * (2*df.J + 1)
    return wt

In [22]:
print(f'Distribution of term "{target}" among levels:')
dflevel[target] = term_distrib(target, dflevel)
display(dflevel.drop(['termwt', 'Composition'], axis=1).sort_values(target, ascending=False).style.format(fmt))
print(f'Total weight of {target} = {dflevel[target].sum():.3f}')

Distribution of term "3F" among levels:


,Lead,J,Jlbl,Erel,Eshift,E,Nr,3F
0,3F,4.0,3F_4,0.0,-1004.9,-1519.421158,"[1, 2, 3, 4, 5, 6, 7, 8, 9]",8.993330
2,3F,3.0,3F_3,1334.1,329.2,-1519.415080,"[17, 18, 19, 20, 21, 22, 23]",6.996944
4,3F,2.0,3F_2,2240.3,1235.3,-1519.410951,"[29, 30, 31, 32, 33]",4.927179
6,(1)1D,2.0,(1)1D_2,4199.1,3194.1,-1519.402026,"[37, 38, 39, 40, 41]",0.024239
3,3D,2.0,3D_2,1727.6,722.7,-1519.413287,"[24, 25, 26, 27, 28]",0.021490
7,3P,2.0,3P_2,14997.0,13992.1,-1519.352827,"[42, 43, 44, 45, 46]",0.015136
9,(2)1D,2.0,(2)1D_2,16885.9,15881.0,-1519.344220,"[50, 51, 52, 53, 54]",0.011956
11,1G,4.0,1G_4,23962.3,22957.4,-1519.311978,"[56, 57, 58, 59, 60, 61, 62, 63, 64]",0.006670
1,3D,3.0,3D_3,1057.4,52.5,-1519.416340,"[10, 11, 12, 13, 14, 15, 16]",0.003056
10,3P,0.0,3P_0,16904.3,15899.4,-1519.344137,[55],0.000000


Total weight of 3F = 21.000


In [23]:
# Summarize level-energy errors by leading term
dftermerr = pd.DataFrame(columns=['Term', 'range', 'mean', 'stds', 'urange', 'umean', 'ustds'])
for term, grp in dfdiff.groupby('uTerm'):
    spread = np.round([grp.err.min(), grp.err.max()], 0).astype(int)
    jwt = 2 * grp.J.values + 1
    m, s = chem.weighted_mean(grp.err, jwt)
    # also consider unsigned (absolute value) errors
    uerr = np.abs(grp.err.values)
    uspread = np.round([uerr.min(), uerr.max()], 0).astype(int)
    um, us = chem.weighted_mean(uerr, jwt)
    dftermerr.loc[len(dftermerr)] = [term, spread, m, s, uspread, um, us]
if dftermerr.isnull().values.any():
    print('*** Some terms are missing ***')
    print('Try decreasing the energy maximum ("termcut")')
else:
    # round values to nearest 1 cm-1
    cols = ['mean', 'stds', 'umean', 'ustds']
    dftermerr[cols] = np.round(dftermerr[cols], 0).astype(int)
    print(f'Molpro source file: {fsoc}')
    print(f'J-weighted errors in level energies (cm-1), grouped by leading term')
    # default order same as experimental terms
    dftermerr.Term = pd.Categorical(dftermerr.Term, xterms)
    dftermerr = dftermerr.sort_values('Term')
dftermerr.sort_values('mean')

Molpro source file: ../UMemphis/Ni15T20S-cc-pVTZ-DK.out
J-weighted errors in level energies (cm-1), grouped by leading term


,Term,range,mean,stds,urange,umean,ustds
5,3F,"[0, 24]",6,6,"[0, 24]",6,6
0,(1)1D,"[789, 789]",789,-1,"[789, 789]",789,-1
4,3D,"[848, 871]",855,4,"[848, 871]",855,4
6,3P,"[887, 1276]",1109,144,"[887, 1276]",1109,144
1,(2)1D,"[1476, 1476]",1476,-1,"[1476, 1476]",1476,-1
2,1G,"[1860, 1860]",1860,-1,"[1860, 1860]",1860,-1
3,1S,"[30716, 30716]",30716,-1,"[30716, 30716]",30716,-1


In [24]:
# Term energy errors as inferred from all levels
dftermerr = pd.DataFrame(columns=['Term', 'wmean', 'wstds'])
termlist = []
wmean = []
wstds = []
# also consider unsigned (absolute value) errors
uwmean = []
uwstds = []
for term in set(dfdiff.uTerm):
    termlist.append(term)
    weights = term_distrib(term, dfdiff).values
    m, s = chem.weighted_mean(dfdiff.err, weights)
    wmean.append(m)
    wstds.append(s)
    uerr = np.abs(dfdiff.err.values)
    um, us = chem.weighted_mean(uerr, weights)
    uwmean.append(um)
    uwstds.append(us)
dftermerr['Term'] = termlist
dftermerr['wmean'] = wmean
dftermerr['wstds'] = wstds
dftermerr['uwmean'] = uwmean
dftermerr['uwstds'] = uwstds

if dftermerr.isnull().values.any():
    print('*** Some terms are missing ***')
    print('Try decreasing the energy maximum ("termcut")')
else:
    print('Errors in term energies (cm-1) as inferred from full level distribution')
    print('\t(not only levels where leading)')
    # default order same as experimental terms
    dftermerr.Term = pd.Categorical(dftermerr.Term, xterms)
    dftermerr = dftermerr.sort_values('Term')
dftermerr.sort_values('wmean').style.format(fmt)

Errors in term energies (cm-1) as inferred from full level distribution
	(not only levels where leading)


,Term,wmean,wstds,uwmean,uwstds
5,3F,10.5,6.7,10.5,6.7
2,(1)1D,793.5,7.2,793.5,7.2
0,3D,850.6,5.2,850.6,5.2
1,3P,1175.5,136.8,1175.5,136.8
6,(2)1D,1366.8,73.5,1366.8,73.5
3,1G,1858.6,2.0,1858.6,2.0
4,1S,30621.8,138.6,30621.8,138.6


In [25]:
print(f'Molpro source file: {fsoc}')
print(f'Alternative values for SOC({target}) of atom {atom}:')
print('-' * 25)
print('{:12s} {:.1f} cm-1'.format('eq (1)', SOC1))
print('{:12s} {:.1f} cm-1'.format('raw theory', SOCraw))
print('{:12s} {:.1f} cm-1'.format('avgd theory', SOCth))
print('{:12s} {:.1f} cm-1'.format('eq (2)', SOC2))
print('-' * 25)

Molpro source file: ../UMemphis/Ni15T20S-cc-pVTZ-DK.out
Alternative values for SOC(3F) of atom Ni:
-------------------------
eq (1)       -971.8 cm-1
raw theory   -1004.9 cm-1
avgd theory  -1004.9 cm-1
eq (2)       -994.4 cm-1
-------------------------
